In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/projects/LLM/AgenticRAG/rag_agents

TODO: Google Drive working directory,

In [1]:
!pipenv install git+https://github.com/stanfordnlp/dspy.git

Mounted at /content/drive
/content/drive/MyDrive/projects/LLM/AgenticRAG/rag_agents


In [5]:
import random

import dspy_util as util
from dotenv import load_dotenv

In [2]:
data = util.load_dataset()
random.shuffle(data)

train = data[:10]
dev = data[10:]
print(f'train: {len(train)}')
print(f'dev: {len(dev)}')
print(type(train[0]))

train: 10
dev: 44
<class 'dspy.primitives.example.Example'>


In [9]:
import sys
[key for key in sys.modules.keys() if 'dspy_util' in key]

['dspy_util']

In [25]:
import importlib
import dspy_util as util
importlib.reload(util)
import dspy_util as util

llm, rm = util.configure_dspy_v1()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\woshi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
rm("What is Jessica’s relationship with Duke Leto?", k=5)[0]

{'id': 'bb89541c-918c-445f-9d9a-671d5e102add',
 'score': 0.3895341348747989,
 'long_text': ' history."\n"Don\'t be facetious, girl! You know as well as I do what forces surround us. We\'ve a three-point civilization: the Imperial Household balanced against the Federated Great Houses of the Landsraad, and between them, the Guild with its damnable monopoly on interstellar transport. In politics, the tripod is the most unstable of all structures. It\'d be bad enough without the complication of a feudal trade culture which turns its back on most science." \nJessica spoke bitterly: "Chips in the path of the flood -- and this chip here, this is the Duke Leto, and this one\'s his son, and this one\'s --"\n"Oh, shut up, girl. You entered this with full knowledge of the delicate edge you walked." \n" \'I am Bene Gesserit: I exist only to serve,\' " Jessica quoted. \n"Truth." the old woman said. "And all we can hope for now is to prevent this from erupting into general conflagration, to salvage 

In [26]:
openai_llm = util.get_openai_few_shot_prompt()

question = """
Question: Who was the actor who played Han Solo in the 1977 movie Star Wars?
- A. Carrie Fisher
- B. Mark Hamil
- C. Harrison Ford
- D. Peter Mayhew
Correct Answer: """
print(openai_llm.invoke({'question': question}))

question = """
Question: What does 6 + 7 sum to?
- A. 13
- B. 90
- C. 700
- D. -10
Correct Answer: """
print(openai_llm.invoke({'question': question}))

C
A


In [41]:
import json
import random

with open("dune_questions.json", "r") as f:
    json_data = json.load(f)
idx_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
import dspy
idx_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
examples = [ dspy.Example(question=ex['question'], 
               choices='\n'.join([f"- {idx_map[idx]}. {opt}" for idx, opt in enumerate(ex['choices'].split('\n'))]),
               answer=ex['truth']).with_inputs("question", "choices")  for ex in json_data['questions']]

random.shuffle(examples)
train = examples[:10]
dev = examples[10:]

In [46]:
class GenerateAnswer(dspy.Signature):
    """The following are multiple choice questions with answers. Return the letter of the correct answer."""

    context  = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField(desc="the question", prefix="Question: ")
    choices  = dspy.InputField(desc="choices to choose answer from")
    answer   = dspy.OutputField(desc="the answer whose value is A, B, C, or D",
                                prefix="Correct Answer: ")

In [48]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question, choices):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question, choices=choices)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [49]:
from dspy.teleprompt import BootstrapFewShot, MIPROv2

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Compile!
mod = RAG()

#teleprompter = MIPROv2(prompt_model=turbo, 
#                     task_model=turbo, 
#                     metric=validate_context_and_answer, 
#                     num_candidates=10, 
#                     init_temperature=1)
#kwargs = dict(num_threads=5, display_progress=True, display_table=0)
#compiled_rag = teleprompter.compile(mod, trainset=train, 
#                                    num_batches=2,
#                                    max_bootstrapped_demos=3, #number of few shoot examples. 
#                                    eval_kwargs=kwargs)

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)
compiled_rag = teleprompter.compile(mod, trainset=train)

 70%|███████   | 7/10 [00:01<00:00,  4.03it/s]


Bootstrapped 4 full traces after 8 examples in round 0.


In [51]:
# Ask any question you like to this simple RAG program.
question = "Which planet do the Atreides settle on after leaving Caladan?"
choices = "- A. Earth\n- B. Arrakis\n- C. Sietech Tabr\n- D. Salusa Secundus"

# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = compiled_rag(question=question, choices=choices)

# Print the contexts and the answer.
print(f"Question: {question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: Which planet do the Atreides settle on after leaving Caladan?
Predicted Answer: B
Retrieved Contexts (truncated): ["A beginning is the time for taking the most delicate care that the balances are correct. This every sister of the Bene Gesserit knows. To begin your study of the life of Muad'Dib, then, take care that...", ' Great Houses of the Landsraad.\n"A popular man arouses the jealousy of the powerful," Hawat had said.\nArrakis -- Dune -- Desert Planet.\nPaul fell asleep to dream of an Arrakeen cavern, silent people a...', ' You\'ll ride upon your own two feet without \'thopter or groundcar or mount."\nAnd Paul had been caught more by her tone -- singsong and wavering -- than by her words.\n"When you live upon Arrakis," she ...']


Evaluate pipeline.

In [19]:
import openai
from dspy.evaluate.evaluate import Evaluate

# Define the signature for automatic assessments.
class Assess(dspy.Signature):
    """Assess the quality of a text answer to a question."""

    assessment_question = dspy.InputField()
    assessment_text =     dspy.InputField()
    assessment_decision = dspy.OutputField(desc="Yes or No")

# Set up the evaluate function. We'll use this many times below.
evaluate_on_duneqa = Evaluate(devset=dev, num_threads=1, display_progress=True, display_table=5)

def llm_metric(gold, pred, trace=None):
    print(f'INPUT: {gold.question} | {gold.answer} | {pred.answer}')
    question, answer, prediction = gold.question, gold.answer, pred.answer
    assessment_prompt = f"The text should answer the question `{question}` with `{answer}`. Does the text answer the question?"
    correct =  dspy.Predict(Assess)(assessment_question=assessment_prompt, assessment_text=prediction)
    correct = correct.assessment_decision.strip().lower() == 'yes'
    score = 1 if correct else 0
    return score

evaluate_on_duneqa(compiled_rag, metric=llm_metric)

Average Metric: 16 / 34  (47.1): 100%|██████████| 34/34 [01:14<00:00,  2.20s/it]


,question,example_answer,context,pred_answer,llm_metric
0,Who trains Lady Jessica?,The Bene Gesserit,"[' and wondered at it.\nJessica spoke without turning. ""Reverend Mother is waiting in my morning room. Please hurry.""\n\nThe Reverend Mother Gaius Helen Mohiam sat in...",Reverend Mother Gaius Helen Mohiam,0
1,What is Jessicaâ€™s relationship with Duke Leto?,She is his concubine,"[' history.""\n""Don\'t be facetious, girl! You know as well as I do what forces surround us. We\'ve a three-point civilization: the Imperial Household balanced against...",Concubine/Partner,0
2,"According to the Fremen, what is a maker?",A sandworm,"[', dust, everything. They can eat flesh off bones and etch the bones to slivers."" \n""Why don\'t they have weather control?""\n""Arrakis has special problems, costs...",Sandworm,✔️ [1]
3,Who is Tuek?,An Arrakeen smuggler,"['YUEH (yu\'e), Wellington (weling-tun), Stdrd 10,082-10,191; medical doctor of the Suk School (grd Stdrd 10,112); md: Wanna Marcus, B.G. (Stdrd 10,092-10,186?); chiefly noted as betrayer...",Fremen leader,0
4,What house does Paul belong to?,House Atreides,"['."" He released the arm, patted Paul\'s shoulder, whirled and strode quickly to the door. \n""Thufir! ""Paul called.\nHawat turned, standing in the open doorway. \n""Don\'t...",House Atreides,✔️ [1]


47.06